Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Инструкция по выполнению проекта
- Загрузите и изучите данные.
- Ответьте на вопрос и обоснуйте решение.
- Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)
    - a. Изменится. Приведите примеры матриц.
    - b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.
- Предложите алгоритм преобразования данных для решения задачи. Обоснуйте, почему качество линейной регрессии не поменяется.
- Запрограммируйте этот алгоритм, применив матричные операции. Проверьте, что качество линейной регрессии из sklearn не отличается до и после преобразования. Примените метрику R2.
### Описание данных
- Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
- Целевой признак: количество страховых выплат клиенту за последние 5 лет.

## 1. Загрузка данных

In [1]:
# Подключим все необходимые библиотеки 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import urllib.request
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error

In [2]:
class DataScience:

    def path_to_files(self, path, link):
        Path('datasets').mkdir(parents=True, exist_ok=True)
        def get_file(file_name, url):
            if not os.path.exists(file_name):
                print(file_name, 'файл не найден, будет загружен из сети')
                _ = urllib.request.urlretrieve(url, file_name)
        urls = {
            'dataset': (path, link)
        }
        [get_file(*urls[k]) for k in urls]
        data = pd.read_csv(urls['dataset'][0])
        return data
    
    def corr_diagram(self, x):
        plt.figure(figsize=(12, 10), dpi=80)
        sns.heatmap(x.corr(), xticklabels=x.corr().columns, yticklabels=x.corr().columns, cmap='RdYlGn', center=0,
                    annot=True)
        plt.title('Диаграмма корреляции', fontsize=22)
        plt.xticks(fontsize=12)
        plt.yticks(fontsize=12)
        plt.show()
        
    def plot_corr(self, df, size=11):
        corr = df.corr()
        fig, ax = plt.subplots(figsize=(size, size))
        ax.matshow(corr)
        plt.xticks(range(len(corr.columns)), corr.columns)
        plt.yticks(range(len(corr.columns)), corr.columns)

In [3]:
#присвоим переменной ds наш класс DataScience
ds = DataScience()

#Загрузим данные
df = ds.path_to_files('insurance.csv','https://code.s3.yandex.net/datasets/insurance.csv')

insurance.csv файл не найден, будет загружен из сети


In [4]:
#Изучим наш датасет
display(df.head())
display(df.tail())
display(df.info())
display(df.describe())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0
4999,1,28.0,40600.0,1,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [5]:
#приведем возраст и зарплату к int типу данных
df['Возраст'] = df['Возраст'].astype(int)
df['Зарплата'] = df['Зарплата'].astype(int)
df.dtypes

Пол                  int64
Возраст              int64
Зарплата             int64
Члены семьи          int64
Страховые выплаты    int64
dtype: object

In [6]:
#Посмотрим на количество дубликатов в таблице
df.duplicated().sum()

153

In [7]:
#Посмотрим на количество пустых значений
df.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

In [8]:
#Избавимся от дубликатов
df= df.drop_duplicates()

## 2. Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

TO-DO: <br>
- Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)
    - a. Изменится. Приведите примеры матриц.
    - b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

Формула расчета весов для линейной регрессии:

$$
w = (X^T X)^{-1} X^T y
$$

Представим новую матрицу признаков $X1$ как произведение старой $X$ на матрицу $P$ (обратимую):

$$
X1 = X * P
$$

Подставим новое значение X1 в формулу $w1$:

$$
w1 = ((X P)^T X P)^{-1} (X P)^T y
$$

Раскроем первое произведение $ (XP)^T $:

$$
w1 = (P^T X^T X P)^{-1} P^T X^T y
$$

Перегруппируем множители в скобках, чтобы это выглядело как произведение трех множителей:

$$
w1 = (P^T (X^T X) P)^{-1} P^T X^T y
$$

Раскроем скобки $ (P^T (X^T X) P)^{-1} $:

$$
w1 = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$

Так как $P$ по условию обратимая, то произведение $(P^T)^{-1} P^T$ равно $E$ (единичная матрица):

$$
w1 = P^{-1} (X^T X)^{-1} E X^T y = P^{-1} (X^T X)^{-1} X^T y
$$

Можно заметить, что справа получилась формула для $w$:

$$
w1 = P^{-1} (X^T X)^{-1} X^T y = P^{-1} w
$$

Подставим новое значение весов  𝑤1  в формулу для предсказаний линейной регрессии.
Предсказания модели расчитываются по формуле:

$$
a = Xw
$$

Подставим в эту формулу значения $X1$ и $w1$ для того, чтобы на их основе вычислить предсказания модели $a1$:

$$
a1 = X1w1 = XPP^{-1} w
$$

Так как $P$ по условию обратимая, то произведение $PP^{-1}$ снова равно $E$:

$$
a1 = XPP^{-1} w = X E w = X w = a
$$

Мы доказали, что предсказания $a1$ для матрицы признаков, умноженных на обратимую матрицу $P$ равны предсказаниям $a$.<br>
Следовательно, отвечая на вопрос изменится ли качество линейной регрессии при умножении признаков на обратимую матрицу - нет, не изменится. В данном случае, параметры линейной регрессии в исходной задаче $w$ и в преобразованной $w1$ связаны следующим образом:

$$
w1 = P^{-1} w
$$

### Вывод:
При умножении признаков на обратимую матрицу, предсказания модели не поменяется, но поменяются веса для новых признаков. Придется переобучать модель на новых признаках.


## 3. Алгоритм преобразования

##### Предложите алгоритм преобразования данных для решения задачи и обоснуем, почему качество линейной регрессии не поменяется.

Чтобы защитить данные и потерять качество модели машинного обучения, нужно умножить признаки на обратимую матрицу $P$. 
Для этого алгоритм преобразования данных должен содержать процесс создания подобной матрицы $P$ и ее умножения на матрицу признаков. 

In [10]:
#Так как по условию задачи эта матрица при умножении на матрицу признаков должна давать матрицу такого же размера
#как исходная, то $P$ должна быть квадратной с размером, равным количеству признаков, в нашем случае их 4.
random_matrix = np.random.randint(100,size = (4, 4))
random_matrix

array([[35, 73, 21, 58],
       [32, 38, 11, 15],
       [58, 58, 50, 83],
       [27, 89, 80, 48]])

In [11]:
#Проверим матрицу на обратимость 
matrix_inverted = np.linalg.inv(random_matrix)
matrix_inverted

array([[-0.02077411,  0.03895485,  0.0116212 , -0.00716634],
       [ 0.01781462,  0.00598633, -0.01620885,  0.00463107],
       [-0.02200024, -0.00151004,  0.00776407,  0.01363015],
       [ 0.01532123, -0.03049501,  0.01057687, -0.0064393 ]])

нам известно, что предсказания модели при преобразовании не поменяются, а значит качество модели тоже не должно меняться, так как качество модели - это производная от предсказаний (которые одинаковы до и после преобразований) и целевого признака.

Подкрепим теоритические рассуждения примером на цифрах из таблицы. Для этого самостоятельно расчитаем вектора предсказаний $a$ и $a1$ по формулам и сравним их между собой.

Сначала выделим признаки и целевой признак:

In [12]:
#Обозначим признаки и целевой признак
features = df.drop('Страховые выплаты', axis = 1)
target = df['Страховые выплаты']
 
#Проведем масштабирование с помощью StandardScaler()
scaler = StandardScaler()
scaler.fit(features)

#Сохраним масштабированные признаки в features_scaled
features_scaled = scaler.transform(features)

In [13]:
#Расчитаем веса $w$ для исходных признаков (масштабированных) по стандартной формуле $w = (X^T X)^{-1} X^T y$
w = np.linalg.inv(features_scaled.T.dot(features_scaled)).dot(features_scaled.T).dot(target)

#Расчитаем вектор предсказаний по исходным признакам по формуле 𝑎=𝑋𝑤
a = features_scaled @ w

#Проведем преобразование признаков:
features_scaled_p = features_scaled @ random_matrix

#Расчитаем веса  𝑤1  для преобразованных признаков по стандартной формуле  𝑤=(𝑋𝑇𝑋)−1𝑋𝑇𝑦
w1 = np.linalg.inv(features_scaled_p.T.dot(features_scaled_p)).dot(features_scaled_p.T).dot(target)

#Расчитаем вектор предсказаний по преобразованным признакам по формуле  𝑎=𝑋𝑤
a1 = features_scaled_p @ w1

#Получим вектор разниц между предсказаниями по исходным признакам  𝑎  и преобразованным  𝑎1 :
diff = a - a1

#Просуммируем все элементы вектора diff, чтобы убедиться, что расхождения минимальны.
diff.sum()

-1.2040108493538426e-13

Исходя из полученных данных мы видим, что сумма расхождений совсем маленькая. Это значит 𝑎 равен 𝑎1. Мы доказали, что преобразование не приводит к разным предсказаниям модели и не ухудшает качество модели.

In [14]:
#Чтобы уж наверняка, округлим элекменты векторов предсказаний и выведем на экран их разницу, если она больше 0.
[print(round(a[i])-round(a1[i])) for i in range(len(a)) if np.abs(round(a[i]) - round(a1[i])) > 0]

[]

Расхождений нет

#### Вывод: 
Мы предложилм алгоритм преобразования данных для решения задачи и обосновали, почему качество линейной регрессии не поменяется.

## 4. Проверка алгоритма

In [15]:
#Создадим модель линейной регрессии для исходных данных и посмотрим ее оценку R2_Score
model = LinearRegression(normalize = True)
model.fit(features, target)
predictions = model.predict(features)
mse = mean_squared_error(target, predictions)
print('MSE Score:', mse)
print('R2_Score на исходных данных: ',r2_score(target, predictions))

MSE Score: 0.1252726382276536
R2_Score на исходных данных:  0.4302010046633359


In [16]:
#Преобразуем признаки умножив на матрицу P и рассчитаем R2_Score
features_matrix = features_scaled @ random_matrix
model.fit(features_matrix, target)
predictions_matrix = model.predict(features_matrix)
mse = mean_squared_error(target, predictions)
print('MSE Score:', mse)
print('R2_Score для преобразованных  признаков: ',r2_score(target, predictions_matrix))

MSE Score: 0.1252726382276536
R2_Score для преобразованных  признаков:  0.4302010046633359


#### Вывод:

Исходя их полученных результатов, можно сказать, что преобразованные данные схожи с результатами исходных данных. 
То есть, мы защитили данные пользователей и при этом не ухудшили качество модели машинного обучения.